In [1]:
import pandas as pd
import cv2
import numpy as np
from scipy.stats import multivariate_normal as norm
import matplotlib.pyplot as plt
from keras.applications import VGG16
from keras.preprocessing.image import ImageDataGenerator
from sklearn.svm import SVC

2023-10-25 10:23:36.116162: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
import tensorflow as tf
def to_grayscale_then_rgb(image):
    image = tf.image.rgb_to_grayscale(image)
    return image

In [15]:
file_train_b_csv='/home/jacobo15defrutos/AVS9/4-Red_Siamesa/Iris_train_basic_seg_list.csv'
df_train_basic= pd.read_csv(file_train_b_csv)
datagen=ImageDataGenerator(rescale=1./255.,preprocessing_function=to_grayscale_then_rgb)
df_train_basic['Label']= df_train_basic['Label'].astype(str)
df_train_basic

,Name,Label
0,465557_OD_INF_SANO.jpg,0
1,331281_OD_SUP_CMV.jpg,1
2,742291_OD_INF_SANO.jpg,0
3,394104_OD_INF_SANO.jpg,0
4,649387_OD_NAS_SANO.jpg,0
...,...,...
116,596480_OI_SUP_SANO.jpg,0
117,781208_OI_INF_SANO.jpg,0
118,331281_OI_INF_SANO.jpg,0
119,649387_OI_NAS_SURV.jpg,1


In [16]:
train_generator = datagen.flow_from_dataframe(
    dataframe=df_train_basic,
    directory=r"/home/jacobo15defrutos/AVS9/Data/saved_seg_class_images/train_basic",
    x_col="Name",
    y_col="Label",
    batch_size=1,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(224, 224))#540,810

Found 121 validated image filenames belonging to 2 classes.


In [17]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224,1))
X_features = base_model.predict_generator(train_generator)
y_labels = train_generator.labels

ValueError: The input must have 3 channels; Received `input_shape=(224, 224, 1)`

In [13]:
X_features.shape


(121, 7, 7, 512)

In [12]:
clf = SVC(gamma='scale',decision_function_shape='ovo',kernel='rbf')
clf.fit(X_features,y_labels)

ValueError: Found array with dim 4. SVC expected <= 2.